# Setup


In [1]:
import pandas as pd
import xlrd # pd.read_excel dependency
import openpyxl # pd.read_excel dependency
import jinja2 # dataframe styling dependency # OPTIONAL, you can delete this statement and just not run the cell that needs it (It's just a display cell)
import numpy as np
from tqdm import tqdm
from warnings import simplefilter
import copy
from itertools import product
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
# Section 1, STEP 4: Update the filename_crosswalk dictionary with the new year as a key and the new filename as a value
filename_crosswalk = {
    2025: "2025 Report Card Public Data Set v1.xlsx",
    2024: "24-RC-Pub-Data-Set.xlsx",
    2023: "23-RC-Pub-Data-Set.xlsx",
    2022: "2022-Report-Card-Public-Data-Set.xlsx",
    2021: "2021-RC-Pub-Data-Set.xlsx",
    2020: "2020-Report-Card-Public-Data-Set.xlsx",
    2019: "2019-Report-Card-Public-Data-Set.xlsx",
    2018: "Report-Card-Public-Data-Set.xlsx",
    2017: "rc17.txt",
    2016: "rc16.txt",
    2015: "rc15.txt",
    2014: "rc14.txt",
    2013: "rc13.txt",
    2012: "rc12.txt",
    2011: "rc11u.txt",
    2010: "rc10.txt",
    2009: "rc09.txt",
    2008: "rc08u.txt"
}

START_YEAR = min(filename_crosswalk.keys())
END_YEAR = max(filename_crosswalk.keys())

assessment_crosswalk = {
    2017: "rc17_assessment.txt",
    2016: "rc16_assessment.txt",
    2015: "rc15-assessment.txt"
}

# Section 1, STEP 5: New demographics
DEMOGRAPHICS = ['Female', 'Male', 'Non-binary', 
       'White', 'Asian', 'Black', 'Latinx',
       'American Indian or Alaska Native',
       'Native Hawaiian or Other Pacific Islander', 'Two or More Races',
       'Middle Eastern or North African',
       'EL', 'Former EL', 'Never EL', 'IEP', 'Children with Disabilities', 'Low Income', 
       'Migrant', 'Homeless', 'Youth in Care', 'Unknown']

In [3]:
DEMO_INVENTORY = pd.read_excel('./Demographic Inventory.xlsx')
DEMO_INVENTORY['Index'] = DEMO_INVENTORY['Index'].astype(int)
DEMO_INVENTORY['Year'] = DEMO_INVENTORY['Year'].astype(int)

## Option Variables


In [4]:
OVERWRITE_INVENTORY=False
ALL_IN_ONE=True
WRITE_TO_FILE=True

# Reading Files and Cleaning


In [5]:
def get_layout_file(short_year, sheet=0):
    if int(short_year) > 12:
        x = "x"
    else:
        x = ""
    if short_year == "12" or short_year == "16" or short_year == "15":
        return pd.read_excel("./data/RC" + short_year + "-layout.xls" + x, header=None, sheet_name=sheet)
    else:
        return pd.read_excel("./data/RC" + short_year + "_layout.xls" + x, header=None, sheet_name=sheet)

In [6]:
def label_proficiency(layout_sheet, y):
    if y == 2015:
        layout_sheet.iloc[11046:11102, 1] = 'SCHOOL'
        layout_sheet.iloc[11131:11187, 1] = 'DISTRICT'
        layout_sheet.iloc[11216:11272, 1] = 'STATE'
        return layout_sheet
    elif y == 2016:
        layout_sheet.iloc[11054:11110, 1] = 'SCHOOL'
        layout_sheet.iloc[11139:11195, 1] = 'DISTRICT'
        layout_sheet.iloc[11224:11280, 1] = 'STATE'
        return layout_sheet
    elif y == 2017:
        layout_sheet.iloc[8113:8169, 1] = 'SCHOOL'
        layout_sheet.iloc[8198:8254, 1] = 'DISTRICT'
        layout_sheet.iloc[8283:8339, 1] = 'STATE'
        return layout_sheet
    else:
        return None

In [7]:
layout_demos = set()
layout_demo_key = {
    "ASIAN" : "Asian",
    "BLACK" : "Black",
    "FEMALE" : "Female",
    "HISPANIC" : "Latinx",
    "IEP" : "IEP",
    "LEP" : "EL",
    "LOW INCOME" : "Low Income",
    "MALE" : "Male",
    "MIGRANT" : "Migrant",
    "MULTIRACIAL" : "Two or More Races",
    "NATIVE AMERICAN" : "American Indian or Alaska Native",
    "NATIVE HAWAIIAN AND OTHERS" : "Native Hawaiian or Other Pacific Islander",
    "TOW OR MORE RACES" : "Two or More Races",
    "TWO OR MORE RACES" : "Two or More Races",
    "WHITE" : "White",
}

def clean_layout_file(layout_file, test=False):
    layout_file = layout_file.rename(
        columns={0: 'Column #', 1: 'Second Qualifier', 2: "Demographic", 5: "Metric"})
    
    layout_demos.update(layout_file['Demographic'].str.strip())
    layout_file["Demographic"] = layout_file['Demographic'].str.strip().replace(
        layout_demo_key)
    sq_mask = ~(layout_file['Second Qualifier'].isnull()) & (
        layout_file['Second Qualifier'].str.strip() != '')
    layout_file.loc[sq_mask, 'Demographic'] = layout_file.loc[sq_mask, 'Demographic'] + \
        ' (' + layout_file.loc[sq_mask, 'Second Qualifier'] + ')'
    layout_file = layout_file.iloc[:, [0, 2, 5]]

    # Drop rows that don't have a column number (header rows for categories)
    layout_file['Column #'] = pd.to_numeric(
        layout_file['Column #'], errors='coerce')
    layout_file = layout_file[layout_file['Column #'].notnull()]
    layout_file['Column #'] = layout_file['Column #'].astype(int)

    # Reset index to column number
    layout_file.index = layout_file['Column #'] - 1
    layout_file.index.name = None

    # Drop Column Number column
    layout_file = layout_file.drop(columns='Column #')

    # Replace demographic keys with Advance Illinois standard
    # also clean up mistakes in demographics
    # This makes it so that the demographic terms used in each year do not need to be tracked
    layout_file['Metric'] = layout_file['Metric'].str.strip()
    layout_file['Demographic'] = layout_file['Demographic'].str.strip()

    # Create mask for all rows with demographics
    mask = ~(layout_file["Demographic"].isnull()) & (
        layout_file["Demographic"] != "ALL") & (layout_file["Demographic"] != "ALL STUDENTS")
    # Combine Metric and Demographic columns
    layout_file.loc[mask, "Metric"] = layout_file.loc[mask, "Metric"].astype(
        str) + " - " + layout_file.loc[mask, "Demographic"].astype(str)

    layout_file['Metric'] = layout_file['Metric'].str.replace(r'\bMEETSS\b', 'MEETS', regex=True)

    return layout_file

In [8]:
layout = {}
layout_assessment = {}
# NOTE: the demographics column may have other notes besides just demographic info

for year in range(2008, 2018):
    s = "{:02d}".format(year - 2000)

    # Grab Column Number, Demographic, and Metric columns
    # Combine two demographic columns if there are two
    layout[year] = get_layout_file(s)
    layout[year] = clean_layout_file(layout[year])
    if year > 2014:
        layout_assessment[year] = get_layout_file(s, 1)
        layout_assessment[year] = label_proficiency(
            layout_assessment[year], year)
        layout_assessment[year] = clean_layout_file(
            layout_assessment[year])
        layout_assessment[year] = layout_assessment[year].iloc[6:]
        layout_assessment[year].index = layout_assessment[year].index + \
            layout[year].index[-1] + 1

        layout[year] = pd.concat((layout[year], layout_assessment[year]))

In [9]:
# Replace demographics in teacher data with Advance Illinois standard
all_teacher_demos=set()
layout_teacher_demo_key = {
    "ASIAN" : "Asian",
    "BLACK" : "Black",
    "FEMALE" : "Female",
    "HISPANIC" : "Latinx",
    "MALE" : "Male",
    "NATIVE AMER" : "American Indian or Alaska Native",
    "NATIVE AMERICAN" : "American Indian or Alaska Native",
    "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER" : "Native Hawaiian or Other Pacific Islander",
    "TWO OR MORE RACES" : "Two or More Races",
    "UNKNOWN RACE" : "Unknown",
    "WHITE" : "White",
}

for year in layout.keys():
    teacher_demos = layout[year]['Metric'].str.extract(r'% (.*) TEACH')[0]
    teacher_demos = teacher_demos.dropna()
    all_teacher_demos.update(teacher_demos)
    
    # This line drops the teacher measurements that aren't relevant to demographic replacement
    teacher_demos = teacher_demos[(
        teacher_demos != 'CLASSES NOT TAUGHT BY HIGHLY QUALIFIED') & (teacher_demos != 'of')] 
    
    layout[year].loc[teacher_demos.index,
                     'Demographic'] = teacher_demos.replace(layout_teacher_demo_key)

    layout[year]['Metric'] = layout[year]['Metric'].str.replace(
        'TEACH ER', 'TEACHER')
    layout[year]['Metric'] = layout[year]['Metric'].str.replace(
        'TEACHER- ', 'TEACHER - ')

    layout[year].loc[teacher_demos.index, 'Metric'] = layout[year].loc[teacher_demos.index, 'Metric'].str.replace(
        r'(% )(.*)( TEACH)', lambda m: m[1] + layout_teacher_demo_key[m[2]] + m[3], regex=True)
    
#pd.Series(list(all_teacher_demos)).sort_values().reset_index(drop=True)

In [10]:
all_enroll_demos = set()
layout_enroll_demo_key = {
    "ASIAN" : "Asian",
    "BLACK" : "Black",
    "HISPANIC" : "Latinx",
    "NATIVE AMERICAN" : "American Indian or Alaska Native",
    "HOMELESS" : "Homeless",
    "I.E.P." : "IEP",
    "L.E.P." : "EL",
    "LOW-INCOME" : "Low Income",
    "MULTIRACIAL/ETHNIC" : "Two or More Races",
    "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER" : "Native Hawaiian or Other Pacific Islander",
    "TWO OR MORE RACES" : "Two or More Races",
    "WHITE" : "White",
}

# Replace demographics in enrollment data with Advance Illinois standard
for year in layout.keys():
    enroll_demos = layout[year]['Metric'].str.extract(r'^\w+ - (.*) %$')[0]
    special_enroll = layout[year]['Metric'].str.extract(
        r'(.*) (?:SCHOOL|DISTRICT|STATE) %$')[0]

    enroll_demos = enroll_demos.dropna()
    special_enroll = special_enroll.dropna()
    all_enroll_demos.update(set(enroll_demos))
    all_enroll_demos.update(set(special_enroll))

    special_enroll = special_enroll[special_enroll.apply(
        lambda x: x in layout_enroll_demo_key.keys())]

    layout[year].loc[enroll_demos.index,
                     'Demographic'] = enroll_demos.replace(layout_enroll_demo_key)
    layout[year].loc[special_enroll.index,
                     'Demographic'] = special_enroll.replace(layout_enroll_demo_key)

    layout[year].loc[enroll_demos.index, 'Metric'] = layout[year].loc[enroll_demos.index, 'Metric'].str.replace(
        r'(^\w+ - )(.*)( %)$', lambda m: m[1] + layout_enroll_demo_key[m[2]] + m[3], regex=True)
    layout[year].loc[special_enroll.index, 'Metric'] = layout[year].loc[special_enroll.index, 'Metric'].str.replace(
        r'(.*)( (?:SCHOOL|DISTRICT|STATE) %$)', lambda m: layout_enroll_demo_key[m[1]] + m[2], regex=True)

#pd.Series(list(all_enroll_demos)).sort_values().reset_index(drop=True).replace(layout_enroll_demo_key)

In [11]:
crosswalk = pd.read_excel(
    'Local Historic Crosswalk.xlsx', sheet_name='Name Crosswalk')
crosswalk.index = crosswalk['Year']
crosswalk = crosswalk.drop(columns='Year')
crosswalk = crosswalk.loc[crosswalk.index.dropna()]

demo_info = pd.read_excel(
    'Local Historic Crosswalk.xlsx', sheet_name='Details')
disagg_info = demo_info.groupby('Metric')['Disaggregated'].max()
# True if index is ever disaggregated, false otherwise

In [12]:
report_card = {}
dropped_finance_rcdts = {}

if 'REPORT_CARD' in globals():
    report_card = copy.deepcopy(REPORT_CARD)
else:
    for year in tqdm(filename_crosswalk.keys()):
        
        if year > 2017:
            wkbk = pd.read_excel(
                "./data/" + filename_crosswalk[year], sheet_name=None, dtype='object')
            wkbk.pop('Revision History', None)
            wkbk.pop('Important Notes', None)

            for k in wkbk.keys():
                wkbk[k][crosswalk.loc[year, 'RCDTS']] = wkbk[k][crosswalk.loc[year, 'RCDTS']].astype(str).str.replace("-", "")
                wkbk[k].loc[wkbk[k][crosswalk.loc[year, 'Type']] == 'Statewide', crosswalk.loc[year, 'RCDTS']] = '650000000800000'

            if year == 2021:
                for k in wkbk.keys():
                    wkbk[k].loc[(wkbk[k][crosswalk.loc[year, 'RCDTS']] == '310458000802001') & (
                        wkbk[k][crosswalk.loc[year, 'Type']] == 'District'), crosswalk.loc[year, 'RCDTS']] = '310458000800000'

            report_card[year] = wkbk['General'].copy()

            for k in filter(lambda x: x not in ['General'], wkbk.keys()):
                if k == "Finance":
                    rcdts_counts = wkbk[k][crosswalk.loc[year, 'RCDTS']].value_counts()
                    duplicate_rcdts = list(rcdts_counts[rcdts_counts > 1].index)
                    dropped_finance_rcdts[year] = duplicate_rcdts
                    wkbk[k] = wkbk[k].loc[~(wkbk[k][crosswalk.loc[year, 'RCDTS']].isin(duplicate_rcdts)),:]
                report_card[year] = pd.merge(
                    report_card[year], wkbk[k], on=crosswalk.loc[year, 'RCDTS'], how='outer', validate="1:1", suffixes=('', f"_{k}"))

        elif year > 2014:
            report_card[year] = pd.read_csv("./data/" + filename_crosswalk[year], sep=";",
                                            header=None, dtype='object')

            report_card_w_assessment = pd.read_csv("./data/" + assessment_crosswalk[year], sep=";",
                                                   header=None, dtype='object').iloc[:, 6:]

            report_card_w_assessment.columns = report_card_w_assessment.columns + \
                layout_assessment[year].index[0] - 6

            report_card[year] = pd.concat(
                (report_card[year], report_card_w_assessment), axis=1).rename(columns=layout[year]['Metric'])
        else:
            report_card[year] = pd.read_csv("./data/" + filename_crosswalk[year], sep=";",
                                            header=None, dtype='object').rename(columns=layout[year]['Metric'])
    REPORT_CARD = copy.deepcopy(report_card)

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [03:37<00:00, 12.10s/it]


In [13]:
s = set()
for i in list(map(lambda x: list(x), dropped_finance_rcdts.values())):
    for j in i:
        s.add(j)
s

{'110238000800001', '310458000802001', '540928000800001'}

# Exploration


In [14]:
# EXPLORATORY CELL
search_string = "growth".upper()

results = []

for year in range(START_YEAR, 2018):
    results.append(pd.Series(layout[year].loc[layout[year]['Metric'].str.upper().str.contains(search_string, regex=False), 'Metric'], name=year).reset_index(drop=True))

for year in range(2018, END_YEAR + 1):
    results.append(pd.Series(report_card[year].columns[report_card[year].columns.str.upper().str.contains(search_string, regex=False)], name=year).reset_index(drop=True))

pd.DataFrame(results)#.to_clipboard()

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,AVERAGE READING GROWTH VALUE - SCHOOL - ALL ST...,AVERAGE READING GROWTH VALUE - DISTRICT - ALL ...,AVERAGE READING GROWTH VALUE - STATE - ALL STU...,AVERAGE MATH GROWTH VALUE - SCHOOL - ALL STUDE...,AVERAGE MATH GROWTH VALUE - DISTRICT - ALL STU...,AVERAGE MATH GROWTH VALUE - STATE - ALL STUDEN...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,AVERAGE READING GROWTH VALUE - SCHOOL - ALL ST...,AVERAGE READING GROWTH VALUE - DISTRICT - ALL ...,AVERAGE READING GROWTH VALUE - STATE - ALL STU...,AVERAGE MATH GROWTH VALUE - SCHOOL - ALL STUDE...,AVERAGE MATH GROWTH VALUE - DISTRICT - ALL STU...,AVERAGE MATH GROWTH VALUE - STATE - ALL STUDEN...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Preprocessing


In [15]:
# Adds all possible demographic categories to each column that has disaggregated data
def add_demo_columns(columns, disagg_data):
    out_columns = []
    for col in columns:
        out_columns.append(col)
        if disagg_data[col]:
            out_columns += list(map(lambda x: col + " - " + x,
                                DEMOGRAPHICS))
    return out_columns

In [16]:
# Section 2, Step 5: Typos and formatting issues
# Replace demographic designations in report cards with standardized ones
for year in range(2018, END_YEAR + 1):
    report_card[year].columns = report_card[year].columns.str.strip()
    report_card[year].columns = report_card[year].columns.str.replace(
        'Non-Binary', 'Non-binary')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Non Binary', 'Non-binary')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Non binary', 'Non-binary')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Black or African American', 'Black')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Hispanic or Latino', 'Latinx')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Hispanic', 'Latinx')
    report_card[year].columns = report_card[year].columns.str.replace(
        'CWD', 'Children with Disabilities')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Hawaiian/Pac Islander', 'Native Hawaiian or Other Pacific Islander')
    report_card[year].columns = report_card[year].columns.str.replace(
        r'\bNative Hawaiian or Other Pacific Island\b', 'Native Hawaiian or Other Pacific Islander')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Am Ind/Alaska Nat', 'American Indian or Alaska Native')
    report_card[year].columns = report_card[year].columns.str.replace(
        'MultiRace', 'Two or More Races')
    report_card[year].columns = report_card[year].columns.str.replace(
        'MENA', 'Middle Eastern or North African')
    report_card[year].columns = report_card[year].columns.str.replace(
        'LowIncome', 'Low Income')
    report_card[year].columns = report_card[year].columns.str.replace(
        r'\bTwo or More Race\b', 'Two or More Races', regex=True)
    report_card[year].columns = report_card[year].columns.str.replace(
        '% Homeless students IAR Mathematics Level 1 - Grade 32', '% Homeless students IAR Mathematics Level 1 - Grade 5')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Homeless students IAR Mathematics Level 1 - Grade 3.1', 'Homeless students IAR Mathematics Level 1 - Grade 5')
    report_card[year].columns = report_card[year].columns.str.replace(
        '% ISA Proficiency - Male %', '% ISA Proficiency - Male')
    report_card[year].columns = report_card[year].columns.str.replace(
        'ELA Growth Percentile - White Growth', 'ELA Growth Percentile - White')
    report_card[year].columns = report_card[year].columns.str.replace(
        'ELA Growth Percentile - Black Growth', 'ELA Growth Percentile - Black')
    report_card[year].columns = report_card[year].columns.str.replace(
        'ELA Growth Percentile - Asian Growth', 'ELA Growth Percentile - Asian')
    report_card[year].columns = report_card[year].columns.str.replace(
        'ELA Growth Percentile - Latinx Growth', 'ELA Growth Percentile - Latinx')
    report_card[year].columns = report_card[year].columns.str.replace(
        'ELA Growth Percentile - Native Hawaiian or Other Pacific Islander Growth', 'ELA Growth Percentile - Native Hawaiian or Other Pacific Islander')
    report_card[year].columns = report_card[year].columns.str.replace(
        'ELA Growth Percentile - American Indian or Alaska Native Growth', 'ELA Growth Percentile - American Indian or Alaska Native')
    report_card[year].columns = report_card[year].columns.str.replace(
        'ELA Growth Percentile - Two or More Races Growth', 'ELA Growth Percentile - Two or More Races')
    report_card[year].columns = report_card[year].columns.str.replace(
        'ELA Growth Percentile - Children with Disabilities Growth', 'ELA Growth Percentile - Children with Disabilities')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Math Growth  Percentile - Native Hawaiian or Other Pacific Islander', 'Math Growth Percentile - Native Hawaiian or Other Pacific Islander')
    report_card[year].columns = report_card[year].columns.str.replace(
        'MathGrowth  Percentile - Native Hawaiian or Other Pacific Islander', 'Math Growth Percentile - Native Hawaiian or Other Pacific Islander')
    report_card[year].columns = report_card[year].columns.str.replace(
        'Math Growth Percentile - Latinx Growth', 'Math Growth Percentile - Latinx')

for year in range(2019, END_YEAR + 1):
    report_card[year] = report_card[year].rename(
        columns={'# ISA Participation - White Count': '# ISA Participation - White'})


report_card[2018] = report_card[2018].rename(
    columns={'Math Participation IEP %.1': 'Math Participation EL %', 'Math Participation Total IEP Count.1': 'Math Participation Total EL Count'})

report_card[2019] = report_card[2019].rename(
    columns={'% Math Participation - IEP.1': '% Math Participation - EL'})

In [17]:
# Add all possible demographic categories to each column with disaggregated data
columns = add_demo_columns(crosswalk.columns, disagg_info)

# Create new object filtering out old columns
new_columns = list(filter(lambda x: x not in crosswalk.columns, columns))

# Create new crosswalk with demographic info
demo_crosswalk = crosswalk.copy()
demo_crosswalk[new_columns] = np.nan

for col in new_columns:
    split = col.split(' - ')
    if len(split) > 2:
        metric = ' - '.join(split[:-1])
        demo = split[-1]
    else:
        metric, demo = split
    demo_formats = demo_info.copy().loc[demo_info['Metric'] == metric, [
        'Year', 'Disaggregation Format', 'Special Format']]

    if (demo in ['IEP', 'EL', 'Low Income', 'Homeless']):
        demo_formats.loc[demo_formats['Special Format'].notnull(
        ), 'Disaggregation Format'] = demo_formats.loc[demo_formats['Special Format'].notnull(), 'Special Format']

    demo_formats = demo_formats.set_index('Year')['Disaggregation Format']

    demo_formats = demo_formats.str.replace('demo', demo)
    demo_formats = demo_formats.str.replace('DEMO', demo)
    demo_crosswalk[col] = demo_formats

master_data = pd.DataFrame(columns=['Year'] + columns)

datasets = {}

In [18]:
def scope_data(rename_vals, year, scope, demo_crosswalk, report_card, dropped_district_columns):
    scoped_rename_vals = rename_vals.drop(
        demo_crosswalk.loc[year, ['School Name', 'School Type']])

    if scope == 'DISTRICT':
        scoped_rename_vals.index = scoped_rename_vals.index.str.replace(
            'SCHOOL', scope)
        scoped_rename_vals.index = [
            demo_crosswalk.loc[year, 'RCDTS']] + list(scoped_rename_vals.index[1:])
        found_columns = [item.replace('SCHOOL', scope) for item in demo_crosswalk.loc[year].dropna(
        ) if item.replace('SCHOOL', scope) in report_card[year].columns]
        found_columns.remove("DISTRICT TYPE NAME")
        found_columns.remove("DISTRICT NAME")
    else:
        scoped_rename_vals.index = scoped_rename_vals.index.str.replace(
            'DISTRICT', scope).str.replace('SCHOOL', scope)
        scoped_rename_vals.index = [
            demo_crosswalk.loc[year, 'RCDTS']] + list(scoped_rename_vals.index[1:])

        found_columns = [item.replace('DISTRICT', scope).replace('SCHOOL', scope) for item in demo_crosswalk.loc[year].dropna(
        ) if item.replace('DISTRICT', scope).replace('SCHOOL', scope) in report_card[year].columns]

    found_columns = [demo_crosswalk.loc[year, 'RCDTS'],
                     'SCHOOL TYPE NAME'] + found_columns

    scoped_data = report_card[year].loc[:, found_columns]
    dropped_district_columns[year] = list(
        set(scoped_rename_vals.index) - set(found_columns))
    dropped_district_columns[year].sort()
    scoped_data = scoped_data.rename(columns=scoped_rename_vals)

    return scoped_data, dropped_district_columns

In [19]:
# STEP 3.5: Add new text metrics to text_columns list
text_columns = ['RCDTS', 'Type', 'District Type', 'School Type', 'School Name','District Name', 
                'City', 'County', 'Title 1 Status', 'Summative Designation', 
                'Summative Designation: Student Group(s)']

def adjust_typing(data, verbose=0, text_columns = text_columns):
    for col in filter(lambda x: x not in text_columns, data.columns):
        if verbose > 10:
            print(col)
        try:
            data[col] = pd.to_numeric(data[col])
        except:
            # STEP 5.4.1: Handle new symbols
            data[col] = data[col].astype(str)
            data[col] = data[col].str.replace(",", "")
            data[col] = data[col].str.replace("*", "")
            data[col] = data[col].str.strip()
            data[col] = data[col].str.replace("Not Provided", "")
            data[col] = data[col].str.replace("nan", '')
            data[col] = pd.to_numeric(data[col])
    return data

In [20]:
demo_inventory = demo_info.sort_values(by=['Index','Year']).reset_index()[['Year','Metric','Disaggregated']]
demo_inventory[['All'] + DEMOGRAPHICS] = False
demo_inventory[demo_inventory['Disaggregated']]
demo_inventory = demo_inventory.set_index(['Year','Metric'])
for i in range(START_YEAR, 2018):
    demo_inventory.loc[(i, 'Type'),:] = False
demo_inventory = demo_inventory.astype(bool)

In [21]:
dropped_columns = {}
dropped_district_columns = {}
dropped_state_columns = {}

for year in tqdm(range(START_YEAR, END_YEAR + 1)):
    # the dropna here drops the columns that are not included in the crosswalk
    # and thus not included in the report card for this year.
    years_columns = demo_crosswalk.loc[year].dropna()
    # swap index and vals for renaming

    rename_vals = pd.Series(years_columns.index.values, index=years_columns)
    # this list comprehension drops any columns that are not found in the report card this year
    # this should drop demographic columns that are not found in this year, such as homeless enrollment
    # counts for 2008 it should not however, drop columns that should be found in the report card.
    # Because it is capable of dropping columns that should be there, the dropped columns are recorded in
    # dropped_columns to be checked later
    found_columns = [
        item for item in years_columns if item in report_card[year].columns]

    data = report_card[year].loc[:, found_columns]
    dropped_columns[year] = list(set(years_columns) - set(found_columns))
    dropped_columns[year].sort()

    data = data.rename(columns=rename_vals)

    # District and state level aggregation for 2008-2017
    if year < 2018:
        data['Type'] = 'School'

        district_data, dropped_district_columns = scope_data(
            rename_vals, year, 'DISTRICT', demo_crosswalk, report_card, dropped_district_columns)
        district_data['RCD'] = district_data['RCDTS'].str[:11]
        district_data = district_data[district_data['SCHOOL TYPE NAME'] != 'CHARTER SCH'].drop(
            columns='SCHOOL TYPE NAME')
        district_data['RCDTS'] = district_data['RCD'] + '0000'
        district_data = district_data.groupby(
            'RCD').max().reset_index(drop=True)
        district_data['Type'] = 'District'

        state_data, dropped_state_columns = scope_data(
            rename_vals, year, 'STATE', demo_crosswalk, report_card, dropped_state_columns)
        state_data = pd.DataFrame(state_data.max()).T.drop(
            columns=['RCDTS', 'SCHOOL TYPE NAME', 'City', 'County'])
        state_data['Type'] = 'Statewide'

        data = pd.concat((data, district_data, state_data),
                         axis=0, ignore_index=True)
        data = data[['RCDTS', 'Type'] +
                    [item for item in data.columns if item not in ['RCDTS', 'Type']]]

    data = adjust_typing(data)
    data['Year'] = year

    metrics = pd.Series(data.columns).str.rsplit(' - ', expand=True, n=1).rename(columns={0:'Metric',1:'Demographic'})
    mask = ~metrics['Demographic'].isin(DEMOGRAPHICS) & ~metrics['Demographic'].isnull()
    metrics.loc[mask, 'Metric'] = metrics.loc[mask,'Metric'] + ' - ' + metrics.loc[mask, 'Demographic']
    metrics.loc[mask, 'Demographic'] = None
    metrics['Demographic'] = metrics['Demographic'].fillna('All')
    metrics['Year'] = year
    metrics = metrics.groupby(['Year','Metric']).agg(list)
    metrics[['All'] + DEMOGRAPHICS] = False
    for i in ['All'] + DEMOGRAPHICS:
        metrics[i] = metrics['Demographic'].apply(lambda x: i in x)
    metrics = metrics.drop(columns='Demographic')
    demo_inventory.loc[metrics.index, ['All'] + DEMOGRAPHICS] = metrics


    datasets[year] = data
demo_inventory.loc[(slice(None),'Year'),'All'] = True

100%|██████████| 18/18 [02:58<00:00,  9.92s/it]


In [22]:
dot_ones = {}
for i in range(START_YEAR, END_YEAR + 1):
    c = report_card[i].columns.astype(str)
    dot_ones[i] = (list(c[c.str.contains('.1', regex=False)]))
dot_ones

{2008: [],
 2009: [],
 2010: [],
 2011: [],
 2012: [],
 2013: [],
 2014: [],
 2015: [],
 2016: [],
 2017: [],
 2018: ['Five Essential Survey Ambitious Instruction.1',
  'General Admin 2016-17 - Dollars.1',
  'All students DLM Mathematics Emerging Grade 3.1'],
 2019: ['Five Essential Survey Ambitious Instruction.1',
  '% All Students (Peer Districts) - Black.1',
  '% All Students (Peer Districts) - Latinx.1',
  '% All Students (Peer Districts) - Asian.1',
  '% All Students (Peer Districts) - Nat Haw/Other Pac Isndr.1',
  '% All Students (Peer Districts) - American Indian or Alaska Native.1',
  '% All Students (Peer Districts) - Two or More Races.1'],
 2020: [],
 2021: [],
 2022: [],
 2023: [],
 2024: [],
 2025: []}

# Final Dataset Creation and Processing


In [23]:
master_data = pd.concat(datasets.values(), ignore_index=True)
print("Master Data columns:",len(master_data.columns))

missing_cols = pd.Series(columns)[~pd.Series(columns).isin(master_data.columns)]
print("Columns missing from master_data because they were never included in a report card:", len(missing_cols), 
      "\n\t(i.e. a deprecated column with a new demographic like a non-binary disaggregation of ISAT scores,\n",
      "\tor a any column with disaggregation, but not one particular disaggregation.)")

new_column_order = ['Year'] + list(pd.Series(columns)[pd.Series(columns).isin(master_data.columns)])
print("Length of columns (should equal the master data columns):", len(new_column_order))

Master Data columns: 4298
Columns missing from master_data because they were never included in a report card: 1962 
	(i.e. a deprecated column with a new demographic like a non-binary disaggregation of ISAT scores,
 	or a any column with disaggregation, but not one particular disaggregation.)
Length of columns (should equal the master data columns): 4298


In [24]:
master_data = master_data.loc[:, new_column_order]
master_data = master_data.apply(
    lambda x: x.str.strip() if x.dtype == 'object' else x)

In [25]:
master_data.loc[master_data['Type'] ==
                'Statewide', 'RCDTS'] = '650000000800000'
# Pre-2018 data fills district data to the school level, but this erases that in keeping with the newer protocol
master_data.loc[master_data['Type'] == 'School', list(master_data.columns[master_data.columns.str.contains(
    "Teacher FTE")]) + ['Pupil Teacher Ratio - Elementary', 'Pupil Teacher Ratio - High School']] = np.nan
master_data.columns = master_data.columns.str.replace(
    'Student Enrollment - ', '% Student Enrollment - ')
master_data.columns = master_data.columns.str.replace(
    'Total Teacher FTE - ', '% Teachers - ')

## Inventory Management


In [26]:
def enrollment_teacher_rename(x):
    if x[1] == 'Student Enrollment':
        return (x[0], '% Student Enrollment')
    elif x[1] == 'Total Teacher FTE':
        return (x[0], '% Teachers')
    else:
        return x

mask = (slice(None),['Student Enrollment', 'Total Teacher FTE'])
demo_inventory.loc[mask,'All'] = False
demo_inventory.index = pd.MultiIndex.from_tuples(list(pd.DataFrame(demo_inventory.index)[0].apply(enrollment_teacher_rename)), names=['Year','Metric'])

In [27]:
for i in range(START_YEAR,END_YEAR + 1):
    demo_inventory.loc[(i, 'Student Enrollment'), :] = False
    demo_inventory.loc[(i, 'Student Enrollment'), 'All'] = True
    demo_inventory.loc[(i, 'Total Teacher FTE'), :] = False
    demo_inventory.loc[(i, 'Total Teacher FTE'), 'All'] = True

demo_inventory = demo_inventory.astype(bool)
demo_inventory = pd.merge(demo_inventory.reset_index(),demo_info[['Year','Metric','Index','Category']],how='left',on=['Year','Metric'])
demo_inventory.loc[demo_inventory['Metric'] == 'Type', 'Category'] = 'Identifier'
type_index = demo_inventory.loc[demo_inventory['Metric'] == 'Type', 'Index'].iloc[0]
demo_inventory.loc[demo_inventory['Metric'] == 'Type', 'Index'] = type_index
demo_inventory.loc[demo_inventory['Metric'] == '% Teachers', 'Category'] = 'Teachers-Admin'
teacher_index = demo_inventory.loc[demo_inventory['Metric'] == 'Total Teacher FTE', 'Index'].iloc[0]
demo_inventory.loc[demo_inventory['Metric'] == '% Teachers', 'Index'] = teacher_index
demo_inventory.loc[demo_inventory['Metric'] == '% Student Enrollment', 'Category'] = 'Enrollment-Attendance'
enrollment_index = demo_inventory.loc[demo_inventory['Metric'] == 'Student Enrollment', 'Index'].iloc[0]
demo_inventory.loc[demo_inventory['Metric'] == '% Student Enrollment', 'Index'] = enrollment_index
demo_inventory['Index'] = demo_inventory['Index'].astype(int)

demo_inventory = demo_inventory[['Index','Category','Metric','Year','Disaggregated','All'] + DEMOGRAPHICS]
demo_inventory = demo_inventory.sort_values(by=['Index','Metric','Year']).reset_index(drop=True)

In [28]:
def dataframe_differences(original,new):
    if new.equals(original):
        print('The original dataframe and the new dataframe are identical.')
        return
    
    if len(original.columns) > len(new.columns):
        print('The new dataframe has fewer columns than the original dataframe.')
        print('It is missing the following columns:', list(set(original.columns) - set(new.columns)))
        return
    elif len(original.columns) < len(new.columns):
        print('The new dataframe has more columns than the original dataframe.')
        print('It has the following extra columns:',list(set(new.columns) - set(original.columns)))
        return
    elif (original.columns != new.columns).any():
        if (original.columns.sort_values() != new.columns.sort_values).any():
            print('The new dataframe and the original have the same column names, but they are not in the same order.')
        else:
            differences = (original.columns != new.columns)
            print('The new dataframe and the original dataframe have different column names. The following shows which columns differ:')
            print('Original dataframe column names:',list(original.columns[differences]))
            print('New      dataframe column names:',list(new.columns[differences]))
        return

    print('The column names are the same, and they are in the same order.')

    if (original.dtypes != new.dtypes).any():
        differences = original.dtypes != new.dtypes
        print('The new dataframe does not have the same dtypes as the original. The following columns have different dtypes:')
        print('Original:')
        display(original.dtypes[differences])
        print('New:')
        display(new.dtypes[differences])

    if original.shape[1] > new.shape[1]:
        print('The new dataframe has fewer rows than the original dataframe.')
    elif original.shape[1] < new.shape[1]:
        print('The new dataframe has more rows than the original dataframe.')
    else:
        print('The new dataframe and the original dataframe have the same number of rows.')
    
    if original.sort_values(by=list(original.columns)).reset_index(drop=True).equals(new.sort_values(by=list(new.columns)).reset_index(drop=True)):
        if original.sort_values(by=list(original.columns)).equals(new.sort_values(by=list(new.columns))):
            print('The new dataframe and the original dataframe have the same rows, but they are shuffled. (Non-Index Agnostic)')
        else:
            print('The new dataframe and the original dataframe have the same rows, but they are shuffled. (Index Agnostic)')
        return
    else:
        print('The sorted dataframes have the following index-agnostic differences in data:')
        display(original.sort_values(by=list(original.columns)).reset_index(drop=True).compare(new.sort_values(by=list(new.columns)).reset_index(drop=True)))
    

In [29]:
# COMPARING THE ACTUAL INVENTORIES
dataframe_differences(DEMO_INVENTORY,demo_inventory)

The new dataframe has more columns than the original dataframe.
It has the following extra columns: ['Never EL', 'Non-binary', 'Middle Eastern or North African', 'Former EL']


In [30]:
# COMPARING THE INVENTORIES WHERE THE CURRENT INVENTORY HAS THE MOST RECENT YEAR ERASED
adding_new_year = True
if adding_new_year:
    dataframe_differences(DEMO_INVENTORY,demo_inventory[demo_inventory['Year'] != END_YEAR].sort_values(by=['Index','Metric','Year']).reset_index(drop=True))

The new dataframe has more columns than the original dataframe.
It has the following extra columns: ['Never EL', 'Non-binary', 'Middle Eastern or North African', 'Former EL']


In [31]:
assertion_string = "the demographic inventory has changed. If you have made updates that should " + \
    "impact the demographic inventory, please manually change the file to reflect the updates. " + \
        "Otherwise, changes made to this file are unexpectedly changing which columns & demographics are included in the file."
assert demo_inventory.equals(DEMO_INVENTORY), assertion_string

AssertionError: the demographic inventory has changed. If you have made updates that should impact the demographic inventory, please manually change the file to reflect the updates. Otherwise, changes made to this file are unexpectedly changing which columns & demographics are included in the file.

In [ ]:
if OVERWRITE_INVENTORY:
    demo_inventory.sort_values(by=['Index','Metric','Year']).to_excel('./Demographic Inventory.xlsx',index=False)

# Data Categorization and Writing to File


In [33]:
def find_columns(starters):
    cols = []
    for col in starters:
        cols += list(master_data.columns[master_data.columns.str.startswith(col)])
    return list(pd.Series(cols).drop_duplicates())

In [34]:
cat_walk = demo_inventory.drop_duplicates(subset=['Metric', 'Category']).groupby(
    ['Category']).agg({'Metric': list})
# Section 4, Step 1: Add new category to list
cat_walk = cat_walk.loc[['Identifier', 'Enrollment-Attendance', 'Student Performance',
                         'CTE', 'Teachers-Admin', 'ACT', 'IAR', 'ISA', 'ISAT', 'PARCC', 'SAT'], :]
cat_walk['Demo Metrics'] = cat_walk['Metric'].apply(find_columns)

In [35]:
def write_to_sheets(master_data, cat_walk, district=False, all_in_one=False):
    if district:
        data = master_data[master_data['Type'] == 'District']
        path = 'Historic RC District Data.xlsx'
    else:
        data = master_data
        path = 'Historic RC Data.xlsx'

    if all_in_one:
        master_data.to_csv('Historic RC Data One Sheet.csv',index=False)
        return

    excel_writer = pd.ExcelWriter(path)

    toc = demo_info.groupby(['Metric', 'Category'])[[
        'Disaggregated', 'Index']].max().reset_index()
    toc = toc.set_index('Category', drop=True).sort_values(by='Index').drop(columns='Index')
    toc = toc.loc[list(cat_walk.index), ['Metric', 'Disaggregated']]
    toc = pd.merge(toc, (data.groupby(['Year']).count() > 1).replace(
        {False: '', True: 'X'}).T, how='left', left_on='Metric', right_index=True)
    toc.loc[toc['Metric'] == 'Year', range(START_YEAR, END_YEAR + 1)] = 'X'

    toc.to_excel(excel_writer, sheet_name='Table of Contents')
    demo_inventory.to_excel(excel_writer, sheet_name='Demographic Inventory')

    for cat in tqdm(cat_walk.index[1:]):
        sheet_data = pd.merge(data[cat_walk.loc['Identifier', 'Metric']],
                              data[cat_walk.loc[cat, 'Demo Metrics']], left_index=True, right_index=True)
        years = sheet_data[['Year'] + cat_walk.loc[cat,
                                                   'Demo Metrics']].groupby(['Year']).count().sum(axis=1).astype(bool)

        year_high = years[years].index.max()
        year_low = years[years].index.min()
        sheet_data.query("Year >= @year_low and Year <= @year_high").to_excel(
            excel_writer, sheet_name=cat, index=False)
    excel_writer.close()

In [36]:
if WRITE_TO_FILE and ALL_IN_ONE:
    write_to_sheets(master_data,cat_walk,district=False,all_in_one=True)

In [ ]:
if WRITE_TO_FILE:
    if 'REPORT_CARD' in globals() or 'report_card' in globals():
        del REPORT_CARD
        del report_card

    write_to_sheets(master_data, cat_walk, district=True)

100%|██████████| 10/10 [01:46<00:00, 10.66s/it]


In [ ]:
if WRITE_TO_FILE:
    write_to_sheets(master_data, cat_walk)